## WORK GROUP 4
### Integrants: 
* Luciana Álvarez
* Rick Chuquilin
* Kenia Moscoso 

In [30]:
using Random
using DataFrames
using Lasso
using GLM, StatsModels



In [31]:
using Distributed

## ORTOGONAL LEARNING

In [34]:
include("../Julia_Notebooks/hdmjl/hdmjl.jl")

In [36]:
n = 100
p = 100
beta =  reshape(( 1 ./ (1:p).^ 2 ),100,1)
gamma = reshape(( 1 ./ (1:p).^ 2 ),100,1)

100×1 Matrix{Float64}:
 1.0
 0.25
 0.1111111111111111
 0.0625
 0.04
 0.027777777777777776
 0.02040816326530612
 0.015625
 0.012345679012345678
 0.01
 0.008264462809917356
 0.006944444444444444
 0.005917159763313609
 ⋮
 0.00012624668602449185
 0.0001234567901234568
 0.00012075836251660427
 0.00011814744801512288
 0.00011562030292519366
 0.00011317338162064282
 0.00011080332409972299
 0.00010850694444444444
 0.00010628122010840685
 0.00010412328196584757
 0.00010203040506070809
 0.0001

In [37]:

sd = 1
X = randn(( n, p ))

D = ( X * gamma ) + reshape(randn(n),n, 1)/4 

# DGP 
Y = D + ( X * beta ) + reshape(randn( n ), n, 1 )


100×1 Matrix{Float64}:
 -3.7418448467687044
  0.4333461938271065
  1.1813183801232725
 -0.09755733454065257
  3.254607260277253
 -0.810631917090707
  0.7238564836126573
  0.1150559343324733
 -1.0989773006909476
 -1.1696693713824662
 -4.042854841191649
 -3.4821992896426788
 -0.8035759588175229
  ⋮
  0.6934902651509713
  0.8705044640126745
 -0.0851264782966662
 -2.5567674358842427
  0.6035640110734501
  2.5064319148919285
  1.1489816398147283
  3.011317950241337
 -0.1802556226381935
  0.035802980187761735
  0.8419857606360746
  1.3174412320380962

In [44]:
D = DataFrame(D)
rename!(D, :x1 => :D)

,D
,Float64
1,-2.41633
2,0.148689
3,0.942787
4,-0.313429
5,1.70705
6,-0.254298
7,0.577787
8,0.996948
9,-0.530846


In [43]:
X = DataFrame(X)
Y =  DataFrame(Y)
rename!(Y, :x1 => :Y)

,Y
,Float64
1,-3.74184
2,0.433346
3,1.18132
4,-0.0975573
5,3.25461
6,-0.810632
7,0.723856
8,0.115056
9,-1.09898


In [40]:
X

,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.51294,-1.32214,-1.31663,-1.80027,0.420349,-2.38617,-0.398834
2,0.436383,0.139765,0.247474,-0.568613,-1.25167,-1.77427,0.386621
3,0.203385,1.01122,0.793857,1.65262,0.304276,-1.1413,-0.00220144
4,-0.101117,0.889577,-0.52685,-0.320042,-1.16566,0.887185,0.697827
5,0.996763,0.29763,0.036947,-0.308642,-1.08028,0.611137,1.35707
6,-0.327777,-0.194303,1.40227,2.09619,0.0538941,-1.03558,1.5353
7,0.334907,0.647801,1.32159,0.7535,-0.483006,0.834628,-0.277918
8,1.26445,0.027627,-0.329335,-0.781433,-0.325373,-1.0701,-0.48213
9,-0.443556,-0.925337,1.17952,0.748262,-1.49774,0.108961,-0.509088


In [45]:
DX = DataFrame(hcat( D , X))
YD = DataFrame(hcat( Y , D))

,Y,D
,Float64,Float64
1,-3.74184,-2.41633
2,0.433346,0.148689
3,1.18132,0.942787
4,-0.0975573,-0.313429
5,3.25461,1.70705
6,-0.810632,-0.254298
7,0.723856,0.577787
8,0.115056,0.996948
9,-1.09898,-0.530846


In [42]:
using LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, Tables, TableOperations, StatsBase, FreqTables, DataFrames
    

In [47]:
r_lasso_estimation = rlasso_arg( DX, Y , nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
coef_array = rlasso(r_lasso_estimation)
#"beta"][2:end ,2]
#SX_IDs  = findall(x-> x != 0,coef_array)

LoadError: MethodError: no method matching DataFrame(::Vector{Vector}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

In [37]:
Pkg.add("rlasso")

LoadError: UndefVarError: Pkg not defined

In [58]:
# In case all X coefficients are zero, then regress Y on D
if sum(SX_IDs) == 0 
    reg = lm( @formula(Y ~ D), YD )
    Naive[i] = GLM.coeftable(reg).cols[1,1][2]
# Otherwise, then regress Y on X and D (but only in the selected coefficients)
elseif sum( SX_IDs ) > 0
    X_D = hcat(Y, D, X[:, SX_IDs ],makeunique=true ) 
    rename!(X_D,:x1 => :Y)
    rename!(X_D,:x1_1 => :D)
    rename!(X_D,:x1_2 => :X)
    reg = lm( @formula(Y ~ D+X),X_D )
    Naive[ i ] = GLM.coeftable(reg).cols[1,1][2]
end

LoadError: UndefVarError: SX_IDs not defined

In [36]:
import rlasso

LoadError: ArgumentError: Package rlasso not found in current path:
- Run `import Pkg; Pkg.add("rlasso")` to install the rlasso package.


In [ ]:

# In both cases we save D coefficient

In [55]:
# Regress residuals. 
regY = rlasso_arg( X , Y , nothing, true, false, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
resY = rlasso(regY)["residuals"]

regD = rlasso_arg( X , D , nothing, true, false, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )
resD = rlasso(regD)["residuals"]

eYeD = convert(DataFrame,hcat( resY , resD))
rename!(eYeD,:x1 => :resY)
rename!(eYeD,:x2 => :resD)

res_reg= lm( @formula(resY ~ resD), eYeD)
Orthogonal[ i ] = GLM.coeftable(reg).cols[1,1][2]

LoadError: OutOfMemoryError()

## DOUBLE LASSO - HETEROGENOUS WAGE EFFECTS

In [48]:
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, 
DataStructures, NamedArrays, PrettyTables, StatsModels, Combinatorics

import CodecBzip2

In [49]:
# Importing .Rdata file

cps2012 = load("../data/cps2012.RData")

Dict{String, Any} with 1 entry:
  "data" => 29217×23 DataFrame. Omitted printing of 17 columns…

In [50]:
keys(cps2012)   # get information from key

KeySet for a Dict{String, Any} with 1 entry. Keys:
  "data"

In [51]:
cps2012 = cps2012["data"]

names(cps2012)

23-element Vector{String}:
 "year"
 "lnw"
 "female"
 "widowed"
 "divorced"
 "separated"
 "nevermarried"
 "hsd08"
 "hsd911"
 "hsg"
 "cg"
 "ad"
 "mw"
 "so"
 "we"
 "exp1"
 "exp2"
 "exp3"
 "exp4"
 "weight"
 "married"
 "ne"
 "sc"

In [52]:
   # couples variables combinations 
    combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)

    # combinations without same couple
    expand_exp(args, deg::ConstantTerm) =
        tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

    StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
        apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [53]:
# Basic model 


reg = @formula(lnw ~ -1 + female + female&(widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) + (widowed +
divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so +
we + exp1 + exp2 + exp3)^2 )


formula_basic = apply_schema(reg, schema(reg, cps2012))


FormulaTerm
Response:
  lnw(continuous)
Predictors:
  0
  female(continuous)
  widowed(continuous)
  divorced(continuous)
  separated(continuous)
  nevermarried(continuous)
  hsd08(continuous)
  hsd911(continuous)
  hsg(continuous)
  cg(continuous)
  ad(continuous)
  mw(continuous)
  so(continuous)
  we(continuous)
  exp1(continuous)
  exp2(continuous)
  exp3(continuous)
  widowed(continuous) & divorced(continuous)
  widowed(continuous) & separated(continuous)
  widowed(continuous) & nevermarried(continuous)
  widowed(continuous) & hsd08(continuous)
  widowed(continuous) & hsd911(continuous)
  widowed(continuous) & hsg(continuous)
  widowed(continuous) & cg(continuous)
  widowed(continuous) & ad(continuous)
  widowed(continuous) & mw(continuous)
  widowed(continuous) & so(continuous)
  widowed(continuous) & we(continuous)
  widowed(continuous) & exp1(continuous)
  widowed(continuous) & exp2(continuous)
  widowed(continuous) & exp3(continuous)
  divorced(continuous) & separated(continuo

In [54]:
coefnames(formula_basic)

("lnw", Any["female", "widowed", "divorced", "separated", "nevermarried", "hsd08", "hsd911", "hsg", "cg", "ad"  …  "female & hsd911", "female & hsg", "female & cg", "female & ad", "female & mw", "female & so", "female & we", "female & exp1", "female & exp2", "female & exp3"])

In [55]:
Y = select(cps2012,:lnw)  # uptcome variable
control = coefnames(formula_basic)[2]  # regresors 
names_col = Symbol.(control)  # string to Symbol to create varaible's name 

136-element Vector{Symbol}:
 :female
 :widowed
 :divorced
 :separated
 :nevermarried
 :hsd08
 :hsd911
 :hsg
 :cg
 :ad
 :mw
 :so
 :we
 ⋮
 Symbol("female & nevermarried")
 Symbol("female & hsd08")
 Symbol("female & hsd911")
 Symbol("female & hsg")
 Symbol("female & cg")
 Symbol("female & ad")
 Symbol("female & mw")
 Symbol("female & so")
 Symbol("female & we")
 Symbol("female & exp1")
 Symbol("female & exp2")
 Symbol("female & exp3")

In [56]:
X = StatsModels.modelmatrix(formula_basic,cps2012)

29217×136 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  22.0  4.84    10.648
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  30.0  9.0     27.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0   0.0  0.0      0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  14.0  1.96     2.744
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0   0.0  0.0      0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0   0.0  0.0      0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0     0.0  0.0   0.0  0.0      0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0   0.0  0.0      0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0   0.0  0.0      0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  15.5  2.4025   3.72388
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  …  0.0  0.0   0.0  0.0      0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0   7.0  0.49     0.343
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0   0.0  0.0      0.0
 ⋮            

In [57]:
X = DataFrame(X, names_col)

,female,widowed,divorced,separated,nevermarried,hsd08,hsd911,hsg
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
# Function to get index of constant columns   

cons_column = []

#recoge todos los valores que tengan varianza 0
for i in 1:size(X,2)
    if var(X[!,i]) == 0
        append!(cons_column  , i)      
    end       
end

In [59]:
# Drop constant columns 

names(X)[cons_column]
select!(X, Not(names(X)[cons_column]))

,female,widowed,divorced,separated,nevermarried,hsd08,hsd911,hsg
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
X

,female,widowed,divorced,separated,nevermarried,hsd08,hsd911,hsg
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# demean function
function desv_mean(a)
    a = Matrix(a)   # dataframe to matrix 
    A = mean(a, dims = 1)
    M = zeros(Float64, size(X,1), size(X,2))
    
    for i in 1:size(a,2)
          M[:,i] = a[:,i] .- A[i]
    end
    
    return M
end    


# Matrix Model & demean

X = DataFrame(desv_mean(X), names(X)) # Dataframe and names 

,female,widowed,divorced,separated,nevermarried,hsd08,hsd911
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.571243,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789
2,0.571243,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,0.977821
3,-0.428757,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789
4,0.571243,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789
5,-0.428757,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789
6,-0.428757,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789
7,-0.428757,-0.00797481,-0.113393,-0.0165999,0.843653,-0.0041072,-0.0221789
8,-0.428757,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789
9,-0.428757,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789


In [62]:
# index to get columns that contains female

index = []

for i in 1:size(X,2)  
        if contains( names(X)[i] , "female")
            append!(index, i)
        end  
end

In [63]:
# Control variables 

W = select(X, Not(names(X)[index]))

,widowed,divorced,separated,nevermarried,hsd08,hsd911,hsg
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,-0.247288
2,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,0.977821,-0.247288
3,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,0.752712
4,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,0.752712
5,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,-0.247288
6,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,0.752712
7,-0.00797481,-0.113393,-0.0165999,0.843653,-0.0041072,-0.0221789,0.752712
8,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,-0.247288
9,-0.00797481,-0.113393,-0.0165999,-0.156347,-0.0041072,-0.0221789,-0.247288


In [64]:
index

16-element Vector{Any}:
   1
 102
 103
 104
 105
 106
 107
 108
 109
 110
 111
 112
 113
 114
 115
 116

In [65]:
table = NamedArray(zeros(16, 2))

j = 0

for i in 1:length(index)

j = j + 1
    
#first step
D = select(X, names(X)[index[i]])
    
D_reg_0  = rlasso_arg( W, D, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )


_resid[!,j] =rlasso(D_reg_0)["residuals"]



# third step
    
#Lasso_HDM = lm(D_resid, Y_resid)

#table[j,1] = GLM.coeftable(Lasso_HDM).cols[5][1]
#table[j,2] = GLM.coeftable(Lasso_HDM).cols[6][1]

    
end 

LoadError: MethodError: no method matching DataFrame(::Vector{Vector}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

In [ ]:
#second step
    
Y_reg_0  = rlasso_arg( W, Y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

Y_resid = rlasso(Y_reg_0)["residuals"]

D_resid[!,j] = reshape(D_resid, length(D_resid), 1)

## DOUBLE LASSO - TESTING THE CONVERGENCE HYPOTHESIS

We provide an additional empirical example of partialling-out based on lasso regretion(Double-Lasso) to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:
  $$
  Y = \beta_1 D +  \beta_2'W + \epsilon.
  $$
  
Specifically, we are interested in how the rates  at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).
  
The relationship is captured by $\beta_1$, the *speed of convergence/divergence*, which measures the speed at which poor countries catch up $(\beta_1< 0)$ or fall behind $(\beta_1> 0)$ rich countries, after controlling for $W$. Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $ \beta_1 <0?$ This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.

The outcome $Y$ is the realized annual growth rate of a country's wealth  (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

In [68]:
growth_read = load("../data/GrowthData.RData")

Dict{String, Any} with 1 entry:
  "GrowthData" => 90×63 DataFrame. Omitted printing of 57 columns…

In [69]:
# Since growth_read is a dictionary, we check if there is a key called "GrowthData", the one we need for our analyze
haskey(growth_read, "GrowthData")

true

In [70]:
growth = growth_read["GrowthData"]
names(growth)

63-element Vector{String}:
 "Outcome"
 "intercept"
 "gdpsh465"
 "bmp1l"
 "freeop"
 "freetar"
 "h65"
 "hm65"
 "hf65"
 "p65"
 "pm65"
 "pf65"
 "s65"
 ⋮
 "secc65"
 "seccm65"
 "seccf65"
 "syr65"
 "syrm65"
 "syrf65"
 "teapri65"
 "teasec65"
 "ex1"
 "im1"
 "xr65"
 "tot1"

In [71]:
y = growth[!, "Outcome"]
y = DataFrame([y], [:y])

,y
,Float64
1,-0.0243358
2,0.100473
3,0.0670515
4,0.0640892
5,0.0279295
6,0.0464074
7,0.0673323
8,0.0209777
9,0.0335512


In [72]:
X = select(growth, Not(["Outcome"]))

,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65
,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,6.59167,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29
2,1,6.82979,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91
3,1,8.89508,0.0,0.204244,0.009186,0.26,0.325,0.201,1.0
4,1,7.56528,0.1997,0.248714,0.03627,0.061,0.07,0.051,1.0
5,1,7.1624,0.174,0.299252,0.037367,0.017,0.027,0.007,0.82
6,1,7.21891,0.0,0.258865,0.02088,0.023,0.038,0.006,0.5
7,1,7.8536,0.0,0.182525,0.014385,0.039,0.063,0.014,0.92
8,1,7.70391,0.2776,0.215275,0.029713,0.024,0.035,0.013,0.69
9,1,9.06346,0.0,0.109614,0.002171,0.402,0.488,0.314,1.0


In [81]:
data = [y X]

,y,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65
,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0243358,1,6.59167,0.2837,0.153491,0.043888,0.007,0.013
2,0.100473,1,6.82979,0.6141,0.313509,0.061827,0.019,0.032
3,0.0670515,1,8.89508,0.0,0.204244,0.009186,0.26,0.325
4,0.0640892,1,7.56528,0.1997,0.248714,0.03627,0.061,0.07
5,0.0279295,1,7.1624,0.174,0.299252,0.037367,0.017,0.027
6,0.0464074,1,7.21891,0.0,0.258865,0.02088,0.023,0.038
7,0.0673323,1,7.8536,0.0,0.182525,0.014385,0.039,0.063
8,0.0209777,1,7.70391,0.2776,0.215275,0.029713,0.024,0.035
9,0.0335512,1,9.06346,0.0,0.109614,0.002171,0.402,0.488


In [74]:
    # couples variables combinations 
    combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)

    # combinations without same couple
    expand_exp(args, deg::ConstantTerm) =
        tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

    StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
        apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [88]:
reg = @formula(y ~ -1 + gdpsh465 + bmp1l + freeop + freetar + h65 + hm65 + hf65 + p65 + pm65 + pf65 + s65 + sm65 + sf65 + fert65 + mort65 + lifee065 + gpop1 + fert1 + mort1 + invsh41 + geetot1 + geerec1 + gde1 + govwb1 + govsh41 + gvxdxe41 + high65 + highm65 + highf65 + highc65 + highcm65 + highcf65 + human65 + humanm65 + humanf65 + hyr65 + hyrm65 + hyrf65 + no65 + nom65 + nof65 + pinstab1 + pop65 + worker65 + pop1565 + pop6565 + sec65 + secm65 + secf65 + secc65 + seccm65 + seccf65 + syr65 + syrm65 + syrf65 + teapri65 + teasec65 + ex1 + im1 + xr65 + tot1)
formula_basic = apply_schema(reg, schema(reg, data))

FormulaTerm
Response:
  y(continuous)
Predictors:
  0
  gdpsh465(continuous)
  bmp1l(continuous)
  freeop(continuous)
  freetar(continuous)
  h65(continuous)
  hm65(continuous)
  hf65(continuous)
  p65(continuous)
  pm65(continuous)
  pf65(continuous)
  s65(continuous)
  sm65(continuous)
  sf65(continuous)
  fert65(continuous)
  mort65(continuous)
  lifee065(continuous)
  gpop1(continuous)
  fert1(continuous)
  mort1(continuous)
  invsh41(continuous)
  geetot1(continuous)
  geerec1(continuous)
  gde1(continuous)
  govwb1(continuous)
  govsh41(continuous)
  gvxdxe41(continuous)
  high65(continuous)
  highm65(continuous)
  highf65(continuous)
  highc65(continuous)
  highcm65(continuous)
  highcf65(continuous)
  human65(continuous)
  humanm65(continuous)
  humanf65(continuous)
  hyr65(continuous)
  hyrm65(continuous)
  hyrf65(continuous)
  no65(continuous)
  nom65(continuous)
  nof65(continuous)
  pinstab1(continuous)
  pop65(continuous)
  worker65(continuous)
  pop1565(continuous)
  pop6

In [89]:
coefnames(formula_basic)

("y", Any["gdpsh465", "bmp1l", "freeop", "freetar", "h65", "hm65", "hf65", "p65", "pm65", "pf65"  …  "seccf65", "syr65", "syrm65", "syrf65", "teapri65", "teasec65", "ex1", "im1", "xr65", "tot1"])

In [91]:
Y = select(data,:y)  # uptcome variable
control = coefnames(formula_basic)[2]  # regresors 
names_col = Symbol.(control)  # string to Symbol to create varaible's name 

61-element Vector{Symbol}:
 :gdpsh465
 :bmp1l
 :freeop
 :freetar
 :h65
 :hm65
 :hf65
 :p65
 :pm65
 :pf65
 :s65
 :sm65
 :sf65
 ⋮
 :secc65
 :seccm65
 :seccf65
 :syr65
 :syrm65
 :syrf65
 :teapri65
 :teasec65
 :ex1
 :im1
 :xr65
 :tot1

In [92]:
X = StatsModels.modelmatrix(formula_basic,data)

90×61 Matrix{Float64}:
 6.59167  0.2837  0.153491  0.043888  …  0.0729  0.0667    0.348  -0.014727
 6.82979  0.6141  0.313509  0.061827     0.094   0.1438    0.525   0.00575
 8.89508  0.0     0.204244  0.009186     0.1741  0.175     1.082  -0.01004
 7.56528  0.1997  0.248714  0.03627      0.1265  0.1496    6.625  -0.002195
 7.1624   0.174   0.299252  0.037367     0.1211  0.1308    2.5     0.003283
 7.21891  0.0     0.258865  0.02088   …  0.0634  0.0762    1.0    -0.001747
 7.8536   0.0     0.182525  0.014385     0.0342  0.0428   12.499   0.009092
 7.70391  0.2776  0.215275  0.029713     0.0864  0.0931    7.0     0.01163
 9.06346  0.0     0.109614  0.002171     0.0594  0.046     1.0     0.008169
 8.15191  0.1484  0.110885  0.028579     0.0524  0.0523    2.119   0.007584
 6.92952  0.0296  0.165784  0.020115  …  0.056   0.0826   11.879   0.086032
 7.23778  0.2151  0.078488  0.011581     0.027   0.0275    1.938   0.007666
 8.11582  0.4318  0.137482  0.026547     0.0804  0.093     0.003   0

In [93]:
X = DataFrame(X, names_col)

,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,pm65
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.59167,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,0.37
2,6.82979,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,1.0
3,8.89508,0.0,0.204244,0.009186,0.26,0.325,0.201,1.0,1.0
4,7.56528,0.1997,0.248714,0.03627,0.061,0.07,0.051,1.0,1.0
5,7.1624,0.174,0.299252,0.037367,0.017,0.027,0.007,0.82,0.85
6,7.21891,0.0,0.258865,0.02088,0.023,0.038,0.006,0.5,0.55
7,7.8536,0.0,0.182525,0.014385,0.039,0.063,0.014,0.92,0.94
8,7.70391,0.2776,0.215275,0.029713,0.024,0.035,0.013,0.69,0.69
9,9.06346,0.0,0.109614,0.002171,0.402,0.488,0.314,1.0,1.0


In [94]:
# Function to get index of constant columns   

cons_column = []

#recoge todos los valores que tengan varianza 0
for i in 1:size(X,2)
    if var(X[!,i]) == 0
        append!(cons_column  , i)      
    end       
end

In [96]:
#Drop constant columns 

names(X)[cons_column]
select!(X, Not(names(X)[cons_column]))

,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,pm65
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.59167,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,0.37
2,6.82979,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,1.0
3,8.89508,0.0,0.204244,0.009186,0.26,0.325,0.201,1.0,1.0
4,7.56528,0.1997,0.248714,0.03627,0.061,0.07,0.051,1.0,1.0
5,7.1624,0.174,0.299252,0.037367,0.017,0.027,0.007,0.82,0.85
6,7.21891,0.0,0.258865,0.02088,0.023,0.038,0.006,0.5,0.55
7,7.8536,0.0,0.182525,0.014385,0.039,0.063,0.014,0.92,0.94
8,7.70391,0.2776,0.215275,0.029713,0.024,0.035,0.013,0.69,0.69
9,9.06346,0.0,0.109614,0.002171,0.402,0.488,0.314,1.0,1.0


In [97]:
# demean function
function desv_mean(a)
    a = Matrix(a)   # dataframe to matrix 
    A = mean(a, dims = 1)
    M = zeros(Float64, size(X,1), size(X,2))
    
    for i in 1:size(a,2)
          M[:,i] = a[:,i] .- A[i]
    end
    
    return M
end    


# Matrix Model & demean

X = DataFrame(desv_mean(X), names(X)) # Dataframe and names 

,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.11123,0.114953,-0.0666109,0.0155539,-0.104556,-0.124156,-0.0812333
2,-0.873113,0.445353,0.0934071,0.0334929,-0.0925556,-0.105156,-0.0752333
3,1.19217,-0.168747,-0.0158579,-0.0191481,0.148444,0.187844,0.118767
4,-0.137631,0.0309533,0.0286121,0.00793594,-0.0505556,-0.0671556,-0.0312333
5,-0.540509,0.00525333,0.0791501,0.00903294,-0.0945556,-0.110156,-0.0752333
6,-0.483997,-0.168747,0.0387631,-0.00745406,-0.0885556,-0.0991556,-0.0762333
7,0.150698,-0.168747,-0.0375769,-0.0139491,-0.0725556,-0.0741556,-0.0682333
8,0.00100345,0.108853,-0.00482694,0.00137894,-0.0875556,-0.102156,-0.0692333
9,1.36056,-0.168747,-0.110488,-0.0261631,0.290444,0.350844,0.231767


In [101]:
index

1-element Vector{Any}:
 1

In [103]:
# Control variables 

W = select(X, Not(names(X)[index]))

,bmp1l,freeop,freetar,h65,hm65,hf65,p65
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.114953,-0.0666109,0.0155539,-0.104556,-0.124156,-0.0812333,-0.603333
2,0.445353,0.0934071,0.0334929,-0.0925556,-0.105156,-0.0752333,0.0166667
3,-0.168747,-0.0158579,-0.0191481,0.148444,0.187844,0.118767,0.106667
4,0.0309533,0.0286121,0.00793594,-0.0505556,-0.0671556,-0.0312333,0.106667
5,0.00525333,0.0791501,0.00903294,-0.0945556,-0.110156,-0.0752333,-0.0733333
6,-0.168747,0.0387631,-0.00745406,-0.0885556,-0.0991556,-0.0762333,-0.393333
7,-0.168747,-0.0375769,-0.0139491,-0.0725556,-0.0741556,-0.0682333,0.0266667
8,0.108853,-0.00482694,0.00137894,-0.0875556,-0.102156,-0.0692333,-0.203333
9,-0.168747,-0.110488,-0.0261631,0.290444,0.350844,0.231767,0.106667


In [104]:
table = NamedArray(zeros(16, 2))

j = 0

for i in 1:length(index)

j = j + 1
    
#first step
D = select(X, names(X)[index[i]])
    
D_reg_0  = rlasso_arg( W, D, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )


D_resid = rlasso(D_reg_0)["residuals"]

#second step
    
Y_reg_0  = rlasso_arg( W, Y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

Y_resid = rlasso(Y_reg_0)["residuals"]

D_resid = reshape(D_resid, length(D_resid), 1)

# third step
    
Lasso_HDM = lm(D_resid, Y_resid)

table[j,1] = GLM.coeftable(Lasso_HDM).cols[5][1]
table[j,2] = GLM.coeftable(Lasso_HDM).cols[6][1]

    
end 

## The same problem persist in the exercise 1, 2 and this one. 

LoadError: MethodError: no method matching DataFrame(::Vector{Vector}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

In [105]:
D = select(X, names(X)[1])
    
D_reg_0  = rlasso_arg( W, D, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )


D_resid = rlasso(D_reg_0)["residuals"]

#second step
    
Y_reg_0  = rlasso_arg( W, Y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

Y_resid = rlasso(Y_reg_0)["residuals"]

D_resid = reshape(D_resid, length(D_resid), 1)

# third step
    
Lasso_HDM = lm(D_resid, Y_resid)

table[1,1] = GLM.coeftable(Lasso_HDM).cols[5][1]
table[1,2] = GLM.coeftable(Lasso_HDM).cols[6][1]

LoadError: MethodError: no method matching DataFrame(::Vector{Vector}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\Kenia\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...